In [2]:
import numpy as np

## Разложение произвольной унитарной матрицы на произведение двухуровневых

Алгоритм был взят из книги ''Quantum Computation and Quantum Information'' Michael A. Nielsen & Isaac L. Chuang, глава 4.5.1. Суть алгоритма заключается в рекурсивном приведении матрицы к единичной, используя свойства унитарных матриц.

Конкретнее:
1. В первом столбце обнулим все значения, кроме первого. Первый приведём к единице
2. Первая строка (за исключением первого элемента) обнулится автоматический из свойств унитарных матриц.
3. Переходим на матрицу размерности меньше на 1 и повторяем алгоритм.
4. Когда доходим до матрицы 2x2, просто домножаем на эрмитово сопряженную ($UU^{\dagger} = I$).
5. В итоге получим набор $U_n$ унитарных двухуровневых матриц, которые удовлетворяют условию $U_{n-1}U_{n-2}...U_1U = I$, где $U$ - первоначальная матрица

In [3]:
# примеры матриц для тестирования
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])
CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
QFT = 1/2 * np.array([[1, 1, 1, 1], [1, complex(0, 1), -1, complex(0, -1)], [1, -1, 1, -1], [1, complex(0, -1), -1, complex(0, 1)]])
F

array([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j ,  0. +0.5j, -0.5+0.j ,  0. -0.5j],
       [ 0.5+0.j , -0.5+0.j ,  0.5+0.j , -0.5+0.j ],
       [ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j]])

In [77]:
# функция для проверки нулевых значений с учетом погрешности машинной точности
def zero_checker(matrix):
    temp_matrix = matrix
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if abs(np.real(matrix[i, j])) < 1e-15:
                temp_matrix[i, j] = complex(0, np.imag(matrix[i, j]))
            if abs(np.imag(matrix[i, j])) < 1e-15:
                temp_matrix[i, j] = complex(np.real(matrix[i, j]), 0)
    return temp_matrix

In [127]:
def unitary_decomposition(matrix, stride = 0):
    """
    Функция декомпозиции. На вход принимает унитарную матрицу, 
    разложение которой нужно найти. Параметр произведения операций
    над матрицей меньших размеров, без изменения размера стартовой матрицы.
    Возвращает список двухуровневых матриц в "правильном" порядке:
    [U_1, U_2, U_3] -> U_3*U_2*U_1*U = I
    """
    column = matrix[:, stride]
    two_level_matrixes = []
    prod_res = matrix
    column_len = len(column)
    # Проверка размерности матрицы. Если матрица два на два - рекурсия заканчивается
    if column_len - stride != 2:
        for i in range(1+stride, column_len):
            if column[i] != complex(0.+0.j):
                abs_sum = np.sqrt(abs(column[i])**2 + abs(column[stride])**2)
                tlm = np.eye(column_len, dtype=complex)
                
                tlm[stride, stride] = np.conj(column[stride])/complex(abs_sum, 0)
                tlm[stride, i] = np.conj(column[i])/complex(abs_sum, 0)
                tlm[i, stride] = column[i]/complex(abs_sum, 0)
                tlm[i, i] = -column[stride]/complex(abs_sum, 0)
                
                two_level_matrixes.append(tlm)
                prod_res = np.matmul(tlm, prod_res)
                column = prod_res[:, stride]
            else:
                # Для последнего элемента алгоритм немного отличается
                if i == column_len - 1:
                    tlm = np.eye(column_len, d_type='complex_')
                    tlm[stride, stride] = np.conj(column[stride])
                    two_level_matrixes.append(tlm)
                    prod_res = np.matmul(tlm, prod_res)
                    column = prod_res[:, stride]
                else:
                    tlm = np.eye(column_len)
                    two_level_matrixes.append(tlm)
            prod_res = zero_checker(prod_res)
        return two_level_matrixes + unitary_decomposition(prod_res, stride + 1)
    else:
        tlm = np.transpose(np.conj(prod_res))
        return [tlm]

In [128]:
test = unitary_decomposition(F)
test

[array([[ 0.70710678-0.j,  0.70710678-0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.70710678+0.j, -0.70710678+0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  1.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          1.        +0.j]]),
 array([[ 0.81649658-0.j,  0.        +0.j,  0.57735027-0.j,
          0.        +0.j],
        [ 0.        +0.j,  1.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.57735027+0.j,  0.        +0.j, -0.81649658+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          1.        +0.j]]),
 array([[ 0.8660254-0.j,  0.       +0.j,  0.       +0.j,  0.5      -0.j],
        [ 0.       +0.j,  1.       +0.j,  0.       +0.j,  0.       +0.j],
        [ 0.       +0.j,  0.       +0.j,  1.       +0.j,  0.       +0.j],
        [ 0.5      +0.j,  0.       +0.j,  0.       +0.j, -0.8660254+0.j]]),
 a

Протестируем вручную с матрицей F

In [136]:
pr1 = np.matmul(test[0],F)
zero_checker(pr1)

array([[ 0.70710678+0.j        ,  0.35355339+0.35355339j,
         0.        +0.j        ,  0.35355339-0.35355339j],
       [ 0.        +0.j        ,  0.35355339-0.35355339j,
         0.70710678+0.j        ,  0.35355339+0.35355339j],
       [ 0.5       +0.j        , -0.5       +0.j        ,
         0.5       +0.j        , -0.5       +0.j        ],
       [ 0.5       +0.j        ,  0.        -0.5j       ,
        -0.5       +0.j        ,  0.        +0.5j       ]])

In [137]:
pr2 = np.matmul(test[1],pr1)
zero_checker(pr2)

array([[ 0.8660254 +0.j        ,  0.        +0.28867513j,
         0.28867513+0.j        ,  0.        -0.28867513j],
       [ 0.        +0.j        ,  0.35355339-0.35355339j,
         0.70710678+0.j        ,  0.35355339+0.35355339j],
       [ 0.        +0.j        ,  0.61237244+0.20412415j,
        -0.40824829+0.j        ,  0.61237244-0.20412415j],
       [ 0.5       +0.j        ,  0.        -0.5j       ,
        -0.5       +0.j        ,  0.        +0.5j       ]])

In [138]:
pr3 = np.matmul(test[2],pr2)
zero_checker(pr3)

array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.35355339-0.35355339j,
         0.70710678+0.j        ,  0.35355339+0.35355339j],
       [ 0.        +0.j        ,  0.61237244+0.20412415j,
        -0.40824829+0.j        ,  0.61237244-0.20412415j],
       [ 0.        +0.j        ,  0.        +0.57735027j,
         0.57735027+0.j        ,  0.        -0.57735027j]])

In [139]:
pr4 = np.matmul(test[3],pr3)
zero_checker(pr4)

array([[1.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.81649658+0.j        ,
        0.        +0.40824829j, 0.40824829+0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.70710678+0.j        , 0.        +0.70710678j],
       [0.        +0.j        , 0.        +0.57735027j,
        0.57735027+0.j        , 0.        -0.57735027j]])

In [140]:
pr5 = np.matmul(test[4],pr4)
zero_checker(pr5)

array([[ 1.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  1.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.70710678+0.j        ,  0.        +0.70710678j],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.70710678+0.j        ,  0.        +0.70710678j]])

In [141]:
pr6 = np.matmul(test[5],pr5)
zero_checker(pr6)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])